**Name**:
Size of GOES array

**Description**:  
Here we will see what is the size of the GOES array after cropping to VIIRS array size.
In addtion, we will see how many of the pixels are non-fire pixels.

**Date created**:  
`202-01-02`

**Author**:  
Asaf Vanunu

---

In [15]:
import os
import ML_toolbox as MLT
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

* read df

In [4]:
for file in os.listdir(os.getcwd()):
    if file.endswith('.csv'):
        NC_VIIRS_df = pd.read_csv(file)
        break

In [5]:
no_4_df_path = r"C:\Users\asaf_rs\Dropbox\Fire_Detection\python_ML_project\create_ML_df\temporal_df\no_4_files.csv"
no_4_df = pd.read_csv(no_4_df_path)

* Now let's load VIIRS and edit it

In [10]:
ML_project_path = "F:\\ML_project"
AOI_list = ["mexico", "east_us"] ## list of AOIs
for dir in os.listdir(ML_project_path): ## loop through the directories in the ML_project folder
    if dir in AOI_list: ## check if the directory is in the AOI_list
        print(f"Now working in {dir} directory")
        AOI_path = os.path.join(ML_project_path, dir) ## get the path of the AOI directory
        for sub_dir in os.listdir(AOI_path): ## loop through the directories in the AOI directory
            if sub_dir == "VIIRS": ## check if the directory is the VIIRS directory
                print(f"Now working in {sub_dir} directory")
                VIIRS_path = os.path.join(AOI_path, sub_dir)
                for sub_VIIRS_dir in os.listdir(VIIRS_path):## loop through the directories in the VIIRS directory
                    if sub_VIIRS_dir == "VIIRS_points": ## check if the directory is the VIIRS_points directory
                        print(f"Now working in {sub_VIIRS_dir} directory")
                        VIIRS_points_path = os.path.join(VIIRS_path, sub_VIIRS_dir) ## get the path of the VIIRS_points directory
                        for file in os.listdir(VIIRS_points_path): ## loop through the files in the VIIRS_points directory
                            if file.endswith(".shp"):
                                print(f"Now working in {file} file")
                                VIIRS_file_path = os.path.join(VIIRS_points_path, file)
                                if dir == "mexico":
                                    mexico_VIIRS_gdf = gpd.read_file(VIIRS_file_path)
                                elif dir == "east_us":
                                    east_us_VIIRS_gdf = gpd.read_file(VIIRS_file_path)
    

Now working in east_us directory
Now working in VIIRS directory
Now working in VIIRS_points directory
Now working in VIIRS_points_east_US.shp file
Now working in mexico directory
Now working in VIIRS directory
Now working in VIIRS_points directory
Now working in VIIRS_points_mexico.shp file


In [11]:
VIIRS = pd.concat([mexico_VIIRS_gdf, east_us_VIIRS_gdf], ignore_index=True)
VIIRS[:5]

,latitude,longitude,Fire_file,Unique_dat,Unique_tim,DATE,TIME,DATE_TIME,FRP,FP_MAD_DT,...,FP_T4,FP_T5,FP_con,FP_con_str,scan_line,grid_sampl,View_Zenit,night/day,region,geometry
0,28.684410,-106.088280,VNP14IMG.A2022001.0906.002.2024075110907.nc,A2022001,0912,2022-01-01,09:12,2022-01-01 09:12,1.660624,-0.422919,...,307.343140,283.351349,8,n,2772,4281,33.149998,Night,mexico,POINT (-106.08828 28.68441)
1,23.640375,-103.911507,VNP14IMG.A2022001.2018.002.2024075110907.nc,A2022001,2024,2022-01-01,20:24,2022-01-01 20:24,10.240311,0.511257,...,341.236176,303.784210,8,n,5708,2173,31.449999,Day,mexico,POINT (-103.91151 23.64038)
2,23.965340,-104.134377,VNP14IMG.A2022001.2018.002.2024075110907.nc,A2022001,2024,2022-01-01,20:24,2022-01-01 20:24,13.236159,0.489004,...,336.169220,302.682037,8,n,5812,2203,30.510000,Day,mexico,POINT (-104.13438 23.96534)
3,24.011993,-104.437927,VNP14IMG.A2022001.2018.002.2024075110907.nc,A2022001,2024,2022-01-01,20:24,2022-01-01 20:24,3.556983,0.610675,...,334.978088,298.806671,8,n,5838,2261,28.699999,Day,mexico,POINT (-104.43793 24.01199)
4,24.008282,-104.437370,VNP14IMG.A2022001.2018.002.2024075110907.nc,A2022001,2024,2022-01-01,20:24,2022-01-01 20:24,15.063047,0.701251,...,339.781769,304.147705,8,n,5837,2261,28.699999,Day,mexico,POINT (-104.43737 24.00828)


In [12]:
## Create a new column called "Fire_index" in the VIIRS dataframe
# this is the normalized difference between the brightness temperature of the two channels I4 and I5
# I4 is 3.74 um and I5 is 11.45 um
VIIRS["Fire_index"] = (VIIRS["FP_T4"] - VIIRS["FP_T5"])/(VIIRS["FP_T4"] + VIIRS["FP_T5"])

* Filter out files that does not have 4 temporal GOES images 

In [7]:
NC_VIIRS_df = NC_VIIRS_df[~np.isin(NC_VIIRS_df["GOES_date_time"], no_4_df["No_4_files"])].reset_index(drop=True)

In [17]:
number_of_non_fire_pixel_list = [] ## number of non fire pixels in each image
image_time_stamp_list = [] ## time stamp of each image
for i in range(len(NC_VIIRS_df)):
    print(f"now working on GOES time stamp: {NC_VIIRS_df['GOES_date_time'].iloc[i]}")
    VIIRS_path = NC_VIIRS_df["VIIRS_file_full_path"].iloc[i]
    FDC_path = NC_VIIRS_df["FDC"].iloc[i]
    MCMI_path = NC_VIIRS_df["MCMI"].iloc[i]
    B7 = MLT.crop_GOES_using_VIIRS(GOES_path=MCMI_path, GOES_band=7, VIIRS_path=VIIRS_path)
    B14 = MLT.crop_GOES_using_VIIRS(GOES_path=MCMI_path, GOES_band=14, VIIRS_path=VIIRS_path)
    FI = (B7.values[0] - B14.values[0])/(B7.values[0] + B14.values[0])
    B7_values = B7.values[0] ## 2D array of the fire mask
    total_pixels = B7_values.shape[0] * B7_values.shape[1] ## total number of pixels in the image
    VIIRS_file = NC_VIIRS_df["VIIRS_file"].iloc[i] ## VIIRS file name
    VIIRS_filter = VIIRS[VIIRS["Fire_file"] == VIIRS_file] ## filter the VIIRS dataframe to get the VIIRS data of the current image
    if len(VIIRS_filter) == 0: ## check if there are no VIIRS data for the current image
        print(f"No VIIRS data for {VIIRS_file}")
        continue ## skip the current image
    else: ## if there are VIIRS data for the current image
        ## rasterize the VIIRS data
        try:
            rasterize_VIIRS = MLT.rasterize_VIIRS(cropped_GOES_image=B7, filter_VIIRS_gdf=VIIRS_filter,
                                      rasterize_type="count", number_of_VIIRS_points=1, VIIRS_band = None)
        ## get the actual fire pixel locations
            GOES_fp_pixel_list = MLT.get_GOES_actual_fire_pixel_locations(GOES_Fire_Index_array=FI, rasterize_VIIRS=rasterize_VIIRS)
            VIIRS_to_kill = MLT.VIIRS_locations_to_kill(rasterize_VIIRS=rasterize_VIIRS)
            GOES_to_kill = MLT.GOES_locations_to_kill(GOES_fp_list=GOES_fp_pixel_list,GOES_Fire_Index_array=FI)
            nan_kill_list = MLT.nan_locations_to_kill(GOES_Fire_Index_array=FI)
            kill_list = GOES_to_kill + VIIRS_to_kill + nan_kill_list ## combine the GOES_to_kill, VIIRS_to_kill and nan_loc_list
            corrected_kill  = list(kill_list for kill_list,_ in itertools.groupby(kill_list)) ## remove duplicates
            number_of_non_fire_pixel = total_pixels - len(corrected_kill) ## number of non fire pixels
            number_of_non_fire_pixel_list.append(number_of_non_fire_pixel) ## append the number of non fire pixels to the list
            image_time_stamp_list.append(NC_VIIRS_df["GOES_date_time"].iloc[i]) ## append the time stamp of the image to the list
            print(f"Number of non fire pixels: {number_of_non_fire_pixel}")
            print(f"{i+1} out of {len(NC_VIIRS_df)} images processed")
        except Exception as e:
            print(f"Error: {e} in {NC_VIIRS_df['GOES_date_time'].iloc[i]}")
            continue
        

now working on GOES time stamp: 2022-01-01 07:31
Number of non fire pixels: 1315901
1 out of 4449 images processed
now working on GOES time stamp: 2022-01-01 09:11
Number of non fire pixels: 814802
2 out of 4449 images processed
now working on GOES time stamp: 2022-01-01 18:46
Number of non fire pixels: 1209370
3 out of 4449 images processed
now working on GOES time stamp: 2022-01-01 20:21
Number of non fire pixels: 485359
4 out of 4449 images processed
now working on GOES time stamp: 2022-01-01 20:31
Number of non fire pixels: 756450
5 out of 4449 images processed
now working on GOES time stamp: 2022-01-02 07:11
Number of non fire pixels: 1386759
6 out of 4449 images processed
now working on GOES time stamp: 2022-01-02 08:46
Number of non fire pixels: 770765
7 out of 4449 images processed
now working on GOES time stamp: 2022-01-02 08:51
Number of non fire pixels: 869129
8 out of 4449 images processed
now working on GOES time stamp: 2022-01-02 18:21
Number of non fire pixels: 899354
9 

AttributeError: 'NoneType' object has no attribute 'values'